### Dataset collected from Kaggle:
https://www.kaggle.com/new-york-city/nyc-parking-tickets?select=Parking_Violations_Issued_-_Fiscal_Year_2015.csv

#### NYC Parking VIolations 2015 Dataset

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
client = Client() #Necessary in cases where task paralleling can be applied for Dask functions
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55666,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.91 GiB
Comm: tcp://192.168.1.105:55695,Total threads: 2
Dashboard: http://192.168.1.105:55697/status,Memory: 1.98 GiB
Nanny: tcp://127.0.0.1:55672,


In [80]:
%%writefile validation.yaml
#Update Validation.yaml file with details of the json file
file_type : csv
file_name : Parking_Violations_Issued_-_Fiscal_Year_2015
columns :
        - SUMMONS_NUMBER
        - PLATE_ID
        - REGISTRATION_STATE
        - PLATE_TYPE
        - ISSUE_DATE
        - VIOLATION_CODE
        - VEHICLE_BODY_TYPE
        - VEHICLE_MAKE
        - ISSUING_AGENCY
        - STREET_CODE_1
        - STREET_CODE_2
        - STREET_CODE_3
        - VEHICLE_EXPIRATION_DATE
        - VIOLATION_LOCATION
        - VIOLATION_PRECINCT
        - ISSUER_PRECINCT
        - ISSUER_CODE
        - ISSUER_COMMAND
        - ISSUER_SQUAD
        - VIOLATION_TIME
        - TIME_FIRST_OBSERVED
        - VIOLATION_COUNTY
        - VIOLATION_IN_FRONT_OF_OR_OPPOSITE
        - HOUSE_NUMBER
        - STREET_NAME
        - INTERSECTING_STREET
        - DATE_FIRST_OBSERVED
        - LAW_SECTION
        - SUB_DIVISION
        - VIOLATION_LEGAL_CODE
        - DAYS_PARKING_IN_EFFECT
        - FROM_HOURS_IN_EFFECT
        - TO_HOURS_IN_EFFECT
        - VEHICLE_COLOR
        - UNREGISTERED_VEHICLE
        - VEHICLE_YEAR
        - METER_NUMBER
        - FEET_FROM_CURB
        - VIOLATION_POST_CODE
        - VIOLATION_DESCRIPTION
        - NO_STANDING_OR_STOPPING_VIOLATION
        - HYDRANT_VIOLATION
        - DOUBLE_PARKING_VIOLATION
        - LATITUDE
        - LONGITUDE
        - COMMUNITY_BOARD
        - COMMUNITY_COUNCIL
        - CENSUS_TRACT
        - BIN
        - BBL
        - NTA
datatype:
        {
        'Meter Number' : object,
        'Time First Observed' : object,
        'Violation Location' : object,
        'Violation Legal Code' : object,
        'Double Parking Violation': object,
        'Hydrant Violation': object,
        'No Standing or Stopping Violation': object,
        'Feet From Curb': 'float64',
        'Law Section': 'float64',
        'Vehicle Year': 'float64', 
        'Issuer Squad' : object
        }

Overwriting validation.yaml


In [81]:
#Load Configuration File & Confirm content
import yaml
with open('validation.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
config

{'file_type': 'csv',
 'file_name': 'Parking_Violations_Issued_-_Fiscal_Year_2015',
 'columns': ['SUMMONS_NUMBER',
  'PLATE_ID',
  'REGISTRATION_STATE',
  'PLATE_TYPE',
  'ISSUE_DATE',
  'VIOLATION_CODE',
  'VEHICLE_BODY_TYPE',
  'VEHICLE_MAKE',
  'ISSUING_AGENCY',
  'STREET_CODE_1',
  'STREET_CODE_2',
  'STREET_CODE_3',
  'VEHICLE_EXPIRATION_DATE',
  'VIOLATION_LOCATION',
  'VIOLATION_PRECINCT',
  'ISSUER_PRECINCT',
  'ISSUER_CODE',
  'ISSUER_COMMAND',
  'ISSUER_SQUAD',
  'VIOLATION_TIME',
  'TIME_FIRST_OBSERVED',
  'VIOLATION_COUNTY',
  'VIOLATION_IN_FRONT_OF_OR_OPPOSITE',
  'HOUSE_NUMBER',
  'STREET_NAME',
  'INTERSECTING_STREET',
  'DATE_FIRST_OBSERVED',
  'LAW_SECTION',
  'SUB_DIVISION',
  'VIOLATION_LEGAL_CODE',
  'DAYS_PARKING_IN_EFFECT',
  'FROM_HOURS_IN_EFFECT',
  'TO_HOURS_IN_EFFECT',
  'VEHICLE_COLOR',
  'UNREGISTERED_VEHICLE',
  'VEHICLE_YEAR',
  'METER_NUMBER',
  'FEET_FROM_CURB',
  'VIOLATION_POST_CODE',
  'VIOLATION_DESCRIPTION',
  'NO_STANDING_OR_STOPPING_VIOLATION',
  '

In [4]:
# Re-write Column names on the dataset to standard format & compare against the declared clumns on the yaml file
import re
def validate_columns(dataset, config):
    dataset = dataset.rename(columns = standardize_columns(dataset.columns))
    # dataset = dataset.rename(columns = replace_space_upper(dataset.columns)) 
    #check on missing columns in the dataset. If the dataset has more columns:
    # print(list(dataset.columns))
    if len(dataset.columns) > len(config['columns']):
        config_missing = [x for x in dataset.columns if x not in config['columns']]
        return print('The following columns are not found on the config file: \n{}'.format(config_missing))
    #if config file has more columns:
    elif len(dataset.columns) < len(config['columns']):
        dataset_missing = [x for x in config['columns'] if x not in dataset.columns]
        return print('The following columns are not found on the dataset: \n{}'.format(dataset_missing))
    # If columns are equal, confirm whether the column names are identical
    else:
        config['columns'].sort()
        if list(dataset.columns.sort_values()) != config['columns']:
            return print('The columns are not similar: \n Dataset: \t{} \nConfig File: \t{}'.format(list(dataset.columns.sort_values()), config['columns']))
        else: return('Column validation completed succesfully. All columns accounted for.')


def standardize_columns(features):
    replace_feature = {}
    for feature in features:
        feature_old = feature
        while re.search(' \Z', feature):                #Search for trailing spaces
            feature = re.sub(' \Z','', feature)
            replace_feature[feature_old] = feature      #Update dict with old feature name, & updated feature name
        
        if re.search('\d', feature):                    #Search for digit value & add '_' before it.
            val_prev_dig = re.search('\d', feature).start() - 1 
            if feature[val_prev_dig] != '_':
                feature = feature[:val_prev_dig + 1] + '_' + feature[val_prev_dig + 1:]
        
        replace_feature[feature_old] = re.sub('\W', '', feature.upper().replace(' ','_'))   #Replace non-words with blank, ' ' with '_', & uppercase
        

    return replace_feature


In [82]:
#Load the dataset

parking_violation_df = dd.read_csv('datasets/{}.{}'.format(config['file_name'], config['file_type']),
                                        dtype =  config['datatype'])
        
parking_violation_df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
npartitions=45,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int64,object,object,object,int64,int64,int64,object,object,int64,int64,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,object,object,object,object,float64,float64,object,float64,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [48]:
#Compute the DataFrames details: Shape
# [rows, columns] = client.compute(parking_violation_df.shape)
[rows, columns]

[<Future: pending, key: int-42b98f7e-c070-4fbf-8757-d89feeb813b1>, 51]

In [49]:
[rows.result(), columns]

[11809233, 51]

In [7]:
#Validate columns in dataset against expected in yaml config file.
validate_columns(parking_violation_df, config)

'Column validation completed succesfully. All columns accounted for.'

# End of Assignment

## Personal trial Notes

In [8]:
# %%writefile validation.yaml
# #Update yaml file for test scenario
# file_type : json
# file_name : imdb-part-02
# columns :
#           - REVIEW ID
#           - REVIEWER
#           - MOVIE
#           - RATING
#           - REVIEW SUMMARY
#           - REVIEW DATE
#           - SPOILER TAG
#           - REVIEW DETAIL
#           - HELPFUL

In [9]:
# # Teast 1: Fail: Missing columns in config file
# import random
# imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
# #Adding columns to dataset
# imdb_part2 = imdb_part2.assign(New_Column = lambda x : random.randint(0,1000),
#                                 New_Column2 = lambda x : random.randint(2000,20000))
# with open('validation.yaml', 'r') as stream:
#     config = yaml.safe_load(stream)
# imdb_part2

In [10]:
# # Test 2: Fail: Missing columns in dataset.
# imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
# #Remove columns from dataset
# imdb_part2 = imdb_part2.drop(['review_date', 'helpful'], axis = 1)
# with open('validation.yaml', 'r') as stream:
#     config = yaml.safe_load(stream)
# imdb_part2

In [11]:
# # Test Case 3: Should be Successful
# #Load Test file, update yaml config file in memory, & validate columns
# imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
# with open('validation.yaml', 'r') as stream:
#     config = yaml.safe_load(stream)
# imdb_part2